In [1]:
#import sys 
#!{sys.executable} -m pip install spotipy

import pandas as pd
from dotenv import load_dotenv, dotenv_values
import json 
import os 
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import concurrent.futures
import time 
from spotipy.exceptions import SpotifyException
import numpy as np
import datetime as dt 

In [2]:
load_dotenv()

True

In [3]:
scope = "user-library-read playlist-read-private playlist-read-collaborative"
sp = spotipy.Spotify(auth_manager = SpotifyOAuth(scope=scope))
rundate = str(dt.date.today())

In [ ]:
def get_user_tracks(user): # need to modify to include the year the track was relesed (for filtering for 2013)
    results = user.current_user_saved_tracks(limit=50)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
        time.sleep(1)
    my_songs_df = pd.DataFrame(tracks)

    song_titles = [value.get('name') for value in my_songs_df['track'].values]
    song_release = [track['album']['release_date'] for track in my_songs_df['track']]
    song_popularity = [value.get('popularity') for value in my_songs_df['track'].values]
    song_ids = [value.get('id') for value in my_songs_df['track'].values]
    song_duration = [round(value.get('duration_ms')/60000, 2) for value in my_songs_df['track'].values]
    extracts = pd.DataFrame({'song_name': song_titles, 'song_id':song_ids, 'song_popularity': song_popularity, 'song_release': song_release, 'song_duration': song_duration}).reindex()


    return extracts

def get_user_top_tracks(user):
   results = sp.current_user_top_tracks(limit=50, time_range="long_term")
   tracks = results['items']
   while results['next']:
     results = sp.next(results)
     tracks.extend(results['items'])
     time.sleep(1)
   top_tracks = pd.DataFrame(tracks)[['id', 'name', 'popularity']]
   return top_tracks
    
    
valid_playlists = []

def check_playlist(playlist_id): 
    
    try: 
        results = sp.playlist_tracks(playlist_id) # fetch tracks 
        valid_playlists.append(playlist_id)
        print(f"Valid playlist: {playlist_id}")

    except spotipy.exceptions.SpotifyException as e: 
        print(f"Invalid Playlist: {playlist_id}")
    time.sleep(1)


def get_playlist_tracks(playlist):
    results = sp.playlist_tracks(playlist, limit=50)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    playlist_songs_df = pd.DataFrame(tracks)

    song_titles = [value.get('name') for value in playlist_songs_df['track'].values]
    song_ids = [value.get('id') for value in playlist_songs_df['track'].values]
    extracts = pd.DataFrame({'song_names': song_titles, 'song_ids':song_ids}).reindex()
    
    return extracts


# Might need to speed up the functions

In [5]:
my_songs = get_user_tracks(sp)
my_songs = my_songs.astype({'song_popularity': int})

In [7]:
user_popularity_score = np.average(my_songs['song_popularity'])
# getting the tracks a user most frequently listens to 
top_tracks = get_user_top_tracks(sp)
most_listened_pop_score = round(np.average(top_tracks['popularity'][0:100]),2)
med_song_duration = round(np.average(my_songs['song_duration'][0:100]),2)

In [8]:
d = {"run_date": [rundate], 
     "total_songs": [len(my_songs['song_name'])], 
     "popularity_score": [user_popularity_score], 
     "most_listened_pop_score": [most_listened_pop_score], 
     "med_song_duration": [med_song_duration]}

addon = pd.DataFrame(d)

In [9]:
addon.to_csv(f"daily_dumps/song_stats_{rundate}.csv", index=False)

In [10]:
old = pd.read_csv("daily_dumps/song_stats.csv")

new = pd.concat([old, addon])

new.to_csv("daily_dumps/song_stats.csv", index=False)